# 🏥 Medical Sentiment Model Training

**Fine-tune DistilBERT for Medical Conversation Sentiment Analysis**

This notebook trains a sentiment classifier with 3 classes:
- **Anxious**: Patient expressing worry, fear, concern
- **Neutral**: Factual statements, questions
- **Reassured**: Relief, gratitude, positive outlook

**Author:** Himanshu Sharma

---

## 1️⃣ Setup & Installation

In [ ]:
# Install dependencies
!pip install transformers datasets accelerate scikit-learn matplotlib seaborn --quiet

print("✅ Dependencies installed!")

In [ ]:
import json
import random
import numpy as np
import pandas as pd
import torch
from collections import Counter
from typing import Dict, List

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
    classification_report
)

import matplotlib.pyplot as plt
import seaborn as sns

# Set seeds
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Using: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

## 2️⃣ Load Training Data (220+ Examples)

In [ ]:
# Complete training dataset with balanced classes
SENTIMENT_DATA = [
    # === ANXIOUS (75 examples) ===
    {"text": "I've been experiencing severe headaches for the past week", "label": "anxious"},
    {"text": "The pain is unbearable, I can barely function", "label": "anxious"},
    {"text": "I'm really worried this might be something serious", "label": "anxious"},
    {"text": "Could this be a brain tumor?", "label": "anxious"},
    {"text": "What if the treatment doesn't work?", "label": "anxious"},
    {"text": "I'm scared about the surgery", "label": "anxious"},
    {"text": "My family has a history of cancer, am I at risk?", "label": "anxious"},
    {"text": "The symptoms are getting worse every day", "label": "anxious"},
    {"text": "I can't sleep at night because of the pain", "label": "anxious"},
    {"text": "Is this going to affect my ability to work?", "label": "anxious"},
    {"text": "I've been losing weight without trying", "label": "anxious"},
    {"text": "The numbness in my hands is getting worse", "label": "anxious"},
    {"text": "I'm terrified of the diagnosis", "label": "anxious"},
    {"text": "What are the chances it's cancer?", "label": "anxious"},
    {"text": "I've been having panic attacks more frequently", "label": "anxious"},
    {"text": "The chest pain is really frightening me", "label": "anxious"},
    {"text": "I keep thinking the worst is going to happen", "label": "anxious"},
    {"text": "Will I ever be able to walk normally again?", "label": "anxious"},
    {"text": "The side effects are really bothering me", "label": "anxious"},
    {"text": "I'm afraid the symptoms might come back", "label": "anxious"},
    {"text": "My blood pressure has been dangerously high", "label": "anxious"},
    {"text": "I don't understand why this keeps happening to me", "label": "anxious"},
    {"text": "The dizziness is making it hard to function", "label": "anxious"},
    {"text": "I'm worried I might have a heart attack", "label": "anxious"},
    {"text": "The fatigue is overwhelming", "label": "anxious"},
    {"text": "I can't stop worrying about the test results", "label": "anxious"},
    {"text": "Is there any chance this could be hereditary?", "label": "anxious"},
    {"text": "The pain is affecting my quality of life", "label": "anxious"},
    {"text": "I'm concerned about the long-term effects", "label": "anxious"},
    {"text": "What if the medication causes more problems?", "label": "anxious"},
    {"text": "I've been having trouble remembering things", "label": "anxious"},
    {"text": "The anxiety is making everything worse", "label": "anxious"},
    {"text": "I'm scared to do the procedure", "label": "anxious"},
    {"text": "Will I need surgery?", "label": "anxious"},
    {"text": "The bleeding hasn't stopped", "label": "anxious"},
    {"text": "I'm terrified of needles", "label": "anxious"},
    {"text": "What happens if the condition worsens?", "label": "anxious"},
    {"text": "I've never felt this sick before", "label": "anxious"},
    {"text": "My legs are swelling up", "label": "anxious"},
    {"text": "I'm worried about my children inheriting this", "label": "anxious"},
    {"text": "The pain keeps me awake all night", "label": "anxious"},
    {"text": "I can barely eat because of the nausea", "label": "anxious"},
    {"text": "Is this a sign of something worse?", "label": "anxious"},
    {"text": "I've lost feeling in my feet", "label": "anxious"},
    {"text": "The shortness of breath is scary", "label": "anxious"},
    {"text": "I'm having trouble concentrating", "label": "anxious"},
    {"text": "Will I be able to drive again?", "label": "anxious"},
    {"text": "The tremors are getting worse", "label": "anxious"},
    {"text": "I'm afraid I won't recover", "label": "anxious"},
    {"text": "My vision has been getting blurry", "label": "anxious"},
    {"text": "I'm worried about affording the treatment", "label": "anxious"},
    {"text": "The rash is spreading rapidly", "label": "anxious"},
    {"text": "What if the doctors missed something?", "label": "anxious"},
    {"text": "I've been coughing up blood", "label": "anxious"},
    {"text": "Is the infection getting worse?", "label": "anxious"},
    {"text": "I'm scared I might pass out again", "label": "anxious"},
    {"text": "The joint pain is debilitating", "label": "anxious"},
    {"text": "I can't hold anything without pain", "label": "anxious"},
    {"text": "My throat has been closing up", "label": "anxious"},
    {"text": "I'm worried about the anesthesia", "label": "anxious"},
    {"text": "The itching is driving me crazy", "label": "anxious"},
    {"text": "I can't bend my knee anymore", "label": "anxious"},
    {"text": "Will I need to be hospitalized?", "label": "anxious"},
    {"text": "The swelling hasn't gone down", "label": "anxious"},
    {"text": "I'm having severe abdominal pain", "label": "anxious"},
    {"text": "What if it's appendicitis?", "label": "anxious"},
    {"text": "I've been running a high fever", "label": "anxious"},
    {"text": "The pain shoots down my arm", "label": "anxious"},
    {"text": "I'm afraid of becoming dependent on medication", "label": "anxious"},
    {"text": "My hearing has been getting worse", "label": "anxious"},
    {"text": "I'm worried about the recovery time", "label": "anxious"},
    {"text": "The bruising looks concerning", "label": "anxious"},
    {"text": "I can barely walk up stairs", "label": "anxious"},
    {"text": "Is this allergic reaction dangerous?", "label": "anxious"},
    {"text": "I've been having nightmares about the diagnosis", "label": "anxious"},

    # === NEUTRAL (75 examples) ===
    {"text": "I'm taking my medication as prescribed", "label": "neutral"},
    {"text": "The dosage seems appropriate", "label": "neutral"},
    {"text": "I've been following the diet recommendations", "label": "neutral"},
    {"text": "My appointment is scheduled for next week", "label": "neutral"},
    {"text": "The physical therapy sessions are ongoing", "label": "neutral"},
    {"text": "I understand the treatment plan", "label": "neutral"},
    {"text": "The symptoms have been stable", "label": "neutral"},
    {"text": "I've been tracking my blood pressure daily", "label": "neutral"},
    {"text": "The exercises are manageable", "label": "neutral"},
    {"text": "I'll need to schedule a follow-up", "label": "neutral"},
    {"text": "The medication has some side effects", "label": "neutral"},
    {"text": "I understand the risks involved", "label": "neutral"},
    {"text": "My diet has been consistent", "label": "neutral"},
    {"text": "The test is scheduled for tomorrow", "label": "neutral"},
    {"text": "I've been resting as advised", "label": "neutral"},
    {"text": "The symptoms appear and disappear", "label": "neutral"},
    {"text": "I'm due for a checkup", "label": "neutral"},
    {"text": "The treatment requires daily attention", "label": "neutral"},
    {"text": "I've noticed some changes", "label": "neutral"},
    {"text": "The prescription needs to be refilled", "label": "neutral"},
    {"text": "I'm following the recovery protocol", "label": "neutral"},
    {"text": "The wound is healing normally", "label": "neutral"},
    {"text": "I have a question about the dosage", "label": "neutral"},
    {"text": "The readings have been consistent", "label": "neutral"},
    {"text": "I need clarification on the instructions", "label": "neutral"},
    {"text": "My weight has been stable", "label": "neutral"},
    {"text": "The symptoms occur occasionally", "label": "neutral"},
    {"text": "I understand I need to fast before the test", "label": "neutral"},
    {"text": "The pain is manageable with medication", "label": "neutral"},
    {"text": "I've been keeping a symptom diary", "label": "neutral"},
    {"text": "How often should I take this?", "label": "neutral"},
    {"text": "The treatment takes about an hour", "label": "neutral"},
    {"text": "I've been wearing the brace as instructed", "label": "neutral"},
    {"text": "When should I expect results?", "label": "neutral"},
    {"text": "The symptoms are intermittent", "label": "neutral"},
    {"text": "I understand the procedure now", "label": "neutral"},
    {"text": "My energy levels vary throughout the day", "label": "neutral"},
    {"text": "Is there an alternative medication?", "label": "neutral"},
    {"text": "The therapy sessions are twice weekly", "label": "neutral"},
    {"text": "I've been monitoring my glucose levels", "label": "neutral"},
    {"text": "What are the possible interactions?", "label": "neutral"},
    {"text": "The recovery is progressing as expected", "label": "neutral"},
    {"text": "I need to know the next steps", "label": "neutral"},
    {"text": "My sleep patterns have changed", "label": "neutral"},
    {"text": "The condition is being managed", "label": "neutral"},
    {"text": "I've adjusted my lifestyle accordingly", "label": "neutral"},
    {"text": "What should I avoid eating?", "label": "neutral"},
    {"text": "The exercises are part of my routine", "label": "neutral"},
    {"text": "I'm documenting any changes", "label": "neutral"},
    {"text": "How long until I see improvement?", "label": "neutral"},
    {"text": "The medication schedule is clear", "label": "neutral"},
    {"text": "I've been staying hydrated", "label": "neutral"},
    {"text": "Should I continue with the current dose?", "label": "neutral"},
    {"text": "The symptoms are predictable now", "label": "neutral"},
    {"text": "I understand I need to return for monitoring", "label": "neutral"},
    {"text": "My vitals have been checked regularly", "label": "neutral"},
    {"text": "What triggers these symptoms?", "label": "neutral"},
    {"text": "The treatment plan is straightforward", "label": "neutral"},
    {"text": "I've made the dietary changes", "label": "neutral"},
    {"text": "When is my next appointment?", "label": "neutral"},
    {"text": "The condition requires ongoing management", "label": "neutral"},
    {"text": "I've been taking notes on my symptoms", "label": "neutral"},
    {"text": "Can I exercise with this condition?", "label": "neutral"},
    {"text": "The medication timing is important", "label": "neutral"},
    {"text": "I notice the symptoms after meals", "label": "neutral"},
    {"text": "Should I get a second opinion?", "label": "neutral"},
    {"text": "The healing process takes time", "label": "neutral"},
    {"text": "I've been avoiding strenuous activities", "label": "neutral"},
    {"text": "What are the warning signs to watch for?", "label": "neutral"},
    {"text": "My condition is stable for now", "label": "neutral"},
    {"text": "The doctor explained the procedure", "label": "neutral"},
    {"text": "I have some questions about treatment options", "label": "neutral"},
    {"text": "My insurance should cover this", "label": "neutral"},
    {"text": "The lab results are pending", "label": "neutral"},
    {"text": "I need to pick up my prescription", "label": "neutral"},

    # === REASSURED (75 examples) ===
    {"text": "That's such a relief to hear!", "label": "reassured"},
    {"text": "Thank you so much, doctor", "label": "reassured"},
    {"text": "I'm feeling much better already", "label": "reassured"},
    {"text": "The treatment is really working", "label": "reassured"},
    {"text": "I can finally sleep through the night", "label": "reassured"},
    {"text": "My symptoms have improved dramatically", "label": "reassured"},
    {"text": "That's exactly what I was hoping to hear", "label": "reassured"},
    {"text": "I feel like myself again", "label": "reassured"},
    {"text": "The pain is almost completely gone", "label": "reassured"},
    {"text": "I'm so grateful for your help", "label": "reassured"},
    {"text": "The recovery has been faster than expected", "label": "reassured"},
    {"text": "I can move freely without pain now", "label": "reassured"},
    {"text": "The test results came back normal", "label": "reassured"},
    {"text": "I feel so much more confident now", "label": "reassured"},
    {"text": "My energy levels are back to normal", "label": "reassured"},
    {"text": "The medication is working wonders", "label": "reassured"},
    {"text": "I can finally do things I couldn't before", "label": "reassured"},
    {"text": "Thank goodness it's nothing serious", "label": "reassured"},
    {"text": "I'm feeling hopeful about my recovery", "label": "reassured"},
    {"text": "The physical therapy has been amazing", "label": "reassured"},
    {"text": "I've regained my strength", "label": "reassured"},
    {"text": "The swelling has gone down completely", "label": "reassured"},
    {"text": "I'm back to my normal routine", "label": "reassured"},
    {"text": "The symptoms have completely disappeared", "label": "reassured"},
    {"text": "I can eat without any problems now", "label": "reassured"},
    {"text": "My blood work came back perfect", "label": "reassured"},
    {"text": "I finally have peace of mind", "label": "reassured"},
    {"text": "The surgery was a complete success", "label": "reassured"},
    {"text": "I'm walking without assistance now", "label": "reassured"},
    {"text": "The treatment exceeded my expectations", "label": "reassured"},
    {"text": "I can breathe easily again", "label": "reassured"},
    {"text": "My quality of life has improved so much", "label": "reassured"},
    {"text": "The follow-up scans look great", "label": "reassured"},
    {"text": "I'm amazed at how well I've healed", "label": "reassured"},
    {"text": "The chronic pain is finally manageable", "label": "reassured"},
    {"text": "I can play with my kids again", "label": "reassured"},
    {"text": "The infection has completely cleared", "label": "reassured"},
    {"text": "I feel stronger every day", "label": "reassured"},
    {"text": "My mobility has fully returned", "label": "reassured"},
    {"text": "The results are better than expected", "label": "reassured"},
    {"text": "I can focus clearly again", "label": "reassured"},
    {"text": "The headaches have stopped completely", "label": "reassured"},
    {"text": "I'm sleeping peacefully now", "label": "reassured"},
    {"text": "My appetite has returned to normal", "label": "reassured"},
    {"text": "The wound has healed beautifully", "label": "reassured"},
    {"text": "I can exercise again without problems", "label": "reassured"},
    {"text": "My blood pressure is normal now", "label": "reassured"},
    {"text": "The anxiety has lifted", "label": "reassured"},
    {"text": "I'm back to work full time", "label": "reassured"},
    {"text": "The symptoms haven't returned", "label": "reassured"},
    {"text": "I feel completely healthy again", "label": "reassured"},
    {"text": "The prognosis is excellent", "label": "reassured"},
    {"text": "I'm grateful for the excellent care", "label": "reassured"},
    {"text": "My tests show significant improvement", "label": "reassured"},
    {"text": "The treatment has transformed my life", "label": "reassured"},
    {"text": "I can drive again without issues", "label": "reassured"},
    {"text": "The numbness has completely resolved", "label": "reassured"},
    {"text": "I'm feeling optimistic about the future", "label": "reassured"},
    {"text": "My vision has returned to normal", "label": "reassured"},
    {"text": "The fatigue is completely gone", "label": "reassured"},
    {"text": "I can hear perfectly again", "label": "reassured"},
    {"text": "The allergic reactions have stopped", "label": "reassured"},
    {"text": "I'm feeling fantastic", "label": "reassured"},
    {"text": "The treatment plan worked perfectly", "label": "reassured"},
    {"text": "I'm completely pain-free now", "label": "reassured"},
    {"text": "My condition is in complete remission", "label": "reassured"},
    {"text": "I can't thank you enough", "label": "reassured"},
    {"text": "The recovery was smooth and quick", "label": "reassured"},
    {"text": "I'm finally back to living my life", "label": "reassured"},
    {"text": "Everything has healed perfectly", "label": "reassured"},
    {"text": "The inflammation has completely subsided", "label": "reassured"},
    {"text": "I'm thrilled with the results", "label": "reassured"},
    {"text": "My range of motion is fully restored", "label": "reassured"},
    {"text": "The treatment was a success", "label": "reassured"},
    {"text": "I feel incredible", "label": "reassured"},
]

df = pd.DataFrame(SENTIMENT_DATA)
print(f"📊 Dataset: {len(df)} examples")
print(f"\n   Class distribution:")
for label in df['label'].unique():
    count = (df['label'] == label).sum()
    print(f"   - {label}: {count}")

## 3️⃣ Data Preprocessing & Split

In [ ]:
# Balance classes
min_count = df['label'].value_counts().min()
balanced_dfs = []
for label in df['label'].unique():
    class_df = df[df['label'] == label].sample(n=min_count, random_state=SEED)
    balanced_dfs.append(class_df)
df_balanced = pd.concat(balanced_dfs).sample(frac=1, random_state=SEED).reset_index(drop=True)

# Train/Val/Test split (70/15/15)
train_df, temp_df = train_test_split(df_balanced, test_size=0.3, stratify=df_balanced['label'], random_state=SEED)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=SEED)

print(f"📈 Data Split:")
print(f"   Train: {len(train_df)} ({len(train_df)/len(df_balanced)*100:.0f}%)")
print(f"   Val:   {len(val_df)} ({len(val_df)/len(df_balanced)*100:.0f}%)")
print(f"   Test:  {len(test_df)} ({len(test_df)/len(df_balanced)*100:.0f}%)")

In [ ]:
# Label encoding
LABEL2ID = {"anxious": 0, "neutral": 1, "reassured": 2}
ID2LABEL = {v: k for k, v in LABEL2ID.items()}

train_df['label_id'] = train_df['label'].map(LABEL2ID)
val_df['label_id'] = val_df['label'].map(LABEL2ID)
test_df['label_id'] = test_df['label'].map(LABEL2ID)

# Load tokenizer
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Create datasets
def create_dataset(df):
    return Dataset.from_dict({'text': df['text'].tolist(), 'label': df['label_id'].tolist()})

train_ds = create_dataset(train_df)
val_ds = create_dataset(val_df)
test_ds = create_dataset(test_df)

# Tokenize
def tokenize_fn(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

train_tokenized = train_ds.map(tokenize_fn, batched=True)
val_tokenized = val_ds.map(tokenize_fn, batched=True)
test_tokenized = test_ds.map(tokenize_fn, batched=True)

print("✅ Data tokenized and ready!")

## 4️⃣ Train DistilBERT Model

In [ ]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=3, id2label=ID2LABEL, label2id=LABEL2ID
).to(device)

# Metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1}

# Training arguments
training_args = TrainingArguments(
    output_dir="./medical_sentiment",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_steps=10,
    warmup_ratio=0.1,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

print("🚀 Starting training...\n")
trainer.train()
print("\n✅ Training complete!")

## 5️⃣ Evaluation

In [ ]:
# Test set evaluation
results = trainer.predict(test_tokenized)
preds = np.argmax(results.predictions, axis=-1)

print("📊 Test Results:\n")
print(classification_report(results.label_ids, preds, target_names=['anxious', 'neutral', 'reassured']))

In [ ]:
# Confusion matrix
cm = confusion_matrix(results.label_ids, preds)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Anxious', 'Neutral', 'Reassured'],
            yticklabels=['Anxious', 'Neutral', 'Reassured'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Medical Sentiment - Confusion Matrix')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150)
plt.show()

## 6️⃣ Live Demo

In [ ]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        pred_id = torch.argmax(probs).item()
        confidence = probs[0][pred_id].item()
    return ID2LABEL[pred_id], confidence

# Test examples
examples = [
    "I'm really worried about my symptoms",
    "The medication schedule is clear",
    "Thank you doctor, I feel so much better!",
    "What if this is something serious?",
    "I have my follow-up next Tuesday",
    "The pain is completely gone now",
]

print("🔮 Live Predictions:\n")
for text in examples:
    sentiment, conf = predict(text)
    print(f'📝 "{text}"')
    print(f"   → {sentiment.upper()} ({conf:.1%})\n")

## 7️⃣ Save & Download Model

In [ ]:
# Save model
MODEL_PATH = "./medical_sentiment_production"
model.save_pretrained(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)

# Save metrics
metrics = {
    "accuracy": float(accuracy_score(results.label_ids, preds)),
    "samples": len(df_balanced),
    "model": MODEL_NAME,
    "epochs": 5
}
with open(f"{MODEL_PATH}/metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

print(f"✅ Model saved to: {MODEL_PATH}")
print(f"\n📊 Final Accuracy: {metrics['accuracy']:.1%}")

In [ ]:
# Zip and download
!zip -r medical_sentiment_production.zip medical_sentiment_production/

from google.colab import files
files.download('medical_sentiment_production.zip')
print("📥 Model downloaded!")